# Exploring data with CDF

In [ ]:
from bokeh.layout import gridplot
from bokeh.plotting import figure, output_file, show
from bokeh.io  import output_notebook
from bokeh.models import Range1d, LinearAxis, Hover|Tool
from bokeh.resources import INLINE
output_notebook(INLINE)
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
def make_plot(title, hist, edges, x, cdf):
    p = figure(tit)